<a href="https://colab.research.google.com/github/rdfnk/rdfnk/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Instalação de pacotes necessários:

- **Gradio**: Ferramenta usada para criar interfaces de usuário interativas para modelos de machine learning.
- **Transformers**: Biblioteca que permite o uso de modelos pré-treinados da Hugging Face, como o Zephyr-3B.
- **Accelerate**: Otimiza o desempenho em GPUs e facilita a execução de grandes modelos.

In [1]:
!pip install -q -U gradio transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

### 2. Importando o Tokenizer:

O **AutoTokenizer** é responsável por transformar as entradas textuais do usuário em um formato compreensível pelo modelo de linguagem, convertendo palavras em "tokens", que são números processados pelo modelo.

In [2]:
from transformers import AutoTokenizer

### 3. Carregando o Tokenizer do Zephyr-3B:

Aqui, carregamos o **tokenizer** específico do modelo Zephyr-3B da Stability AI, que já foi treinado para processar e entender entradas textuais.

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    'stabilityai/stablelm-zephyr-3b')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

### 4. Importando o Modelo de Linguagem:

A função **AutoModelForCausalLM** é utilizada para carregar modelos pré-treinados que lidam com tarefas de **Language Modeling**, ou seja, gerar ou prever textos com base no que já foi fornecido.

In [4]:
from transformers import AutoModelForCausalLM

### 5. Carregando o Modelo Zephyr-3B:

Carregamos o modelo Zephyr-3B, que foi treinado para entender e gerar texto. O parâmetro `device_map="auto"` permite que o modelo seja alocado automaticamente em uma GPU, se disponível, ou na CPU, dependendo do ambiente de execução.

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    'stabilityai/stablelm-zephyr-3b',
    trust_remote_code=True,
    device_map="auto")

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### 6. Definindo o Dispositivo de Execução:

Esta célula define se o código será executado em uma **GPU** (se disponível) ou em uma **CPU**, melhorando o desempenho quando uma GPU estiver presente. Isso é feito utilizando a biblioteca **PyTorch**.

In [6]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50304, 2560)
    (layers): ModuleList(
      (0-31): 32 x StableLmDecoderLayer(
        (self_attn): StableLmSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affin

### 7. Definindo a Classe ChatBot:

Aqui, criamos uma classe chamada **ChatBot** que gerencia o histórico de conversas e a função de previsão (responder às entradas do usuário). A função `predict` recebe a entrada do usuário e retorna uma resposta gerada pelo modelo de linguagem.

### Parâmetros para ajustar o comportamento das respostas:

1. **system_prompt**: Define o contexto do chatbot.
   - Exemplo: `"You are a friendly assistant who explains things simply."`

2. **max_new_tokens**: Controla o comprimento das respostas. Ao mudar esse texto, você altera o "contexto" do chatbot, influenciando o tipo de resposta que ele fornecerá. Se você mudar o prompt para algo mais específico, como "You are a casual assistant", o chatbot pode fornecer respostas mais informais.
   - Exemplo: `max_new_tokens=150` para respostas mais curtas.

3. **temperature**: Controla a criatividade das respostas. Valores altos (0.8-1.0) geram mais variação, valores baixos (0.2-0.5) geram respostas mais previsíveis.
   - Exemplo: `temperature=1.0` para mais criatividade.

4. **do_sample**: Se `True`, adiciona aleatoriedade nas respostas. Se `False`, gera respostas mais previsíveis.
   - Exemplo: `do_sample=True` para mais variação.

5. **top_p**: Utiliza "nucleus sampling" para limitar a geração de palavras às mais prováveis até que somem a probabilidade `p`. (Pode ser inserido no trecho de código)
   - Exemplo: `top_p=0.9` para mais variação e fluidez.

6. **repetition_penalty**: Penaliza a repetição de palavras. Valores maiores que 1.0 ajudam a evitar respostas repetitivas. (Pode ser inserido no trecho de código)
   - Exemplo: `repetition_penalty=1.2` para reduzir repetições.

In [7]:
class ChatBot:
  def __init__(self):
    self.history = []
  def predict(self, user_input,
              system_prompt="You are an expert analyst and provide assessment:"):
    prompt = [{'role': 'user', 'content': user_input + "\n" + system_prompt + ":"}]
    inputs = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=True,
        return_tensors='pt',
    )
    tokens = model.generate(
        inputs.to(model.device),
        max_new_tokens=250, # Controla o tamanho das respostas
        temperature=0.8, # Controla a criatividade das respostas
        do_sample=False # Escolhe a resposta mais provável
    )
    response_text = tokenizer.decode(
        tokens[0],
        skip_special_tokens=False)
    del tokens
    torch.cuda.empty_cache()
    return response_text

### 8. Inicializando o ChatBot:

Nesta célula, criamos uma instância do chatbot, que será usada nas próximas etapas para gerar respostas com base nas entradas fornecidas pelo usuário.

In [8]:
bot = ChatBot()

### 9. Importando a Biblioteca Gradio:

A biblioteca **Gradio** é usada para criar uma interface gráfica amigável, permitindo que os usuários interajam com o chatbot diretamente por meio de uma interface web.

In [9]:
import gradio as gr

### 10. Criando a Interface do ChatBot:

Nesta célula, uma interface é criada usando o **Gradio**. A função `bot.predict` será usada para gerar respostas e exemplos de entradas são fornecidas para ajudar o usuário a entender como interagir com o Chatbot. A interface também inclui um título para contextualizar o aplicativo.

### Parâmetros que podem ser alterados:

1. **examples**: Define exemplos pré-carregados para facilitar o uso do chatbot.
   - Exemplo: `examples=[["Pergunta sobre IA?", "Explique como um especialista em IA."]]`

2. **fn**: Função que o Gradio chamará para gerar respostas.
   - Exemplo: `fn=bot.predict` (mantém o mesmo, mas pode ser alterado para outra função se necessário).

3. **title**: Título da interface.
   - Exemplo: `title="Chatbot Educacional"` altera o título da aplicação.

4. **inputs**: Define o tipo de entrada. Pode ser "text", "image", entre outros.
   - Exemplo: `inputs=["text", "text"]` para dois campos de texto.

5. **outputs**: Define o tipo de saída gerada pelo chatbot (texto, imagem, etc.).
   - Exemplo: `outputs="text"` mantém a saída como texto.

6. **iface.launch()**: Inicia a interface Gradio. Parâmetros opcionais, como `share=True`, permitem compartilhar publicamente.
   - Exemplo: `iface.launch(share=True)` para compartilhar o link da interface.

In [10]:
examples = [["What is the dropout technique in machine learning?", "I want you to act as a machine learning engineer. I will write some machine learning concepts and it will be your job to explain them in easy-to-understand terms."]]

iface = gr.Interface(
    fn=bot.predict,
    title="An App with Zephyr-3b",
    examples=examples,
    inputs=["text", "text"],
    outputs="text",
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c1d568ab9cc84a3f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
